In [ ]:
import pandas as pd
import numpy as np
import utils as ut
import os
from matplotlib import pyplot as plt

In [ ]:
FILES_PATH = "./data/"

Even though a notebook is not a viable solution for any porduction-ready code, I decided to use a notebook to implement my code and present the results for the first part of the assignement. It makes things easy to assess, it is easier to comment and discuss results as well as present.

### Import datasets

In [ ]:
fixtures_data = pd.read_csv(f"{FILES_PATH}fixtures.csv", delimiter=",")
odds_data = pd.read_csv(f"{FILES_PATH}odds.csv", delimiter=",")
players_data = pd.read_csv(f"{FILES_PATH}players.csv", delimiter=",")
results_data = pd.read_csv(f"{FILES_PATH}results.csv", delimiter=",")
startingXI_data = pd.read_csv(f"{FILES_PATH}startingXI.csv", delimiter=",")
teams_data = pd.read_csv(f"{FILES_PATH}teams.csv", delimiter=",")

# Data exploration

# Question 1: Which team won the league in the first season?

### Merge results and team names

In [ ]:
results_data = results_data.merge(teams_data, left_on="HomeTeamID", right_on = "TeamID", how="left")\
    .rename(columns = {"TeamName" : "HomeTeamName"})\
        .drop(columns="TeamID")

results_data = results_data.merge(teams_data, left_on="AwayTeamID", right_on = "TeamID", how="left")\
    .rename(columns = {"TeamName" : "AwayTeamName"})\
        .drop(columns="TeamID")

### Add the numbers of points scored for every game
- 3 for a win
- 1 for a draw
- 0 for a loss

In [ ]:
def add_points(score_team1, score_team2):
    if score_team1>score_team2:
        return 3
    elif score_team1<score_team2:
        return 0
    else:
        return 1

In [ ]:
results_data["HomePoints"] = results_data.apply(lambda x: add_points(x.HomeScore, x.AwayScore), axis=1)
results_data["AwayPoints"] = results_data.apply(lambda x: add_points(x.AwayScore, x.HomeScore), axis=1)

In [ ]:
results_data_season1 = results_data[results_data["SeasonID"]==1]

### Create a table with every team for every weekgame as index

In [ ]:
df_home_team = pd.melt(results_data_season1, id_vars = ["HomeTeamID", "HomeTeamName","Gameweek"], value_vars = ["HomePoints"], value_name="PointsScored")\
.rename(columns={"HomeTeamID":"TeamID", "HomeTeamName":"TeamName"})\
.drop(columns="variable")

df_away_team = pd.melt(results_data_season1, id_vars = ["AwayTeamID", "AwayTeamName","Gameweek"], value_vars = ["AwayPoints"], value_name="PointsScored")\
.rename(columns={"AwayTeamID":"TeamID", "AwayTeamName":"TeamName"})\
.drop(columns="variable")

In [ ]:
df_all_team = pd.concat((df_home_team, df_away_team)).sort_values(["Gameweek", "TeamID"])

### Sum all the points scored and get the final table for season 1

In [ ]:
df_ranking_season1 = df_all_team.groupby(["TeamID", "TeamName"]).agg(
    TotalPoints = ("PointsScored", "sum")
).sort_values(by="TotalPoints", ascending=False).reset_index()

df_ranking_season1["Ranking"] = np.arange(1, len(df_ranking_season1)+1)

# Question 2: At what point in the season did that team secure their league title?

According the Premier League official website:


If any clubs finish with the same number of points, their position in the Premier League table is determined by goal difference, then the number of goals scored, then the team who collected the most points in the head-to-head matches, then who scored most away goals in the head-to-head. 

As it is impossible to predict the final goal difference, we will consider that the winner secured their title with certitude when their number of points is higher than the number of points of the 2nd plus the maximum number points left to score. 

### Calculate the number of maximum points left to score for every gameweek

In [ ]:
df_all_team["MaxPointsLeft"] = df_all_team.Gameweek.max()*3 - df_all_team.Gameweek*3

### Calculate the number of total points for every team for every gameweek

In [ ]:
df_all_team["CumulativePoints"] = df_all_team.sort_values(by=["Gameweek", "TeamID"]).groupby(["TeamID", "TeamName"])["PointsScored"].cumsum().values

In [ ]:
df_all_team = df_all_team.merge(df_all_team[df_all_team["TeamID"]==15][["Gameweek", "CumulativePoints"]], on="Gameweek", suffixes=(("", "Winner")))

### Calculate the difference of points between the winning team for each team at every gameweek

In [ ]:
df_all_team["DiffPointsWinner"] = df_all_team["CumulativePointsWinner"] - df_all_team["CumulativePoints"]
df_all_team["DiffPossiblePoints"] = df_all_team.apply(lambda x: 1 if x.DiffPointsWinner>x.MaxPointsLeft else 0, axis = 1)

### Find the first time when the 2nd of the league could not catch up with the winner

In [ ]:
df_all_team["PossibleMaxPoints"] = df_all_team["CumulativePoints"]+df_all_team["MaxPointsLeft"]

In [ ]:
df_plot1 = df_all_team[df_all_team["TeamID"]==8]
df_plot2 = df_all_team[df_all_team["TeamID"]==4]
df_plot3 = df_all_team[df_all_team["TeamID"]==19]
df_plot4 = df_all_team[df_all_team["TeamID"]==5]

x = np.arange(1, len(df_plot1)+1)
plt.plot(x, df_plot1.CumulativePointsWinner)
plt.plot(x, df_plot1.PossibleMaxPoints)
plt.plot(x, df_plot2.PossibleMaxPoints)
plt.plot(x, df_plot3.PossibleMaxPoints)
plt.plot(x, df_plot4.PossibleMaxPoints)

# Question 3: What result was the biggest upset?

In [ ]:
fixtures_data = pd.read_csv(f"{FILES_PATH}fixtures.csv", delimiter=",")
odds_data = pd.read_csv(f"{FILES_PATH}odds.csv", delimiter=",")
players_data = pd.read_csv(f"{FILES_PATH}players.csv", delimiter=",")
results_data = pd.read_csv(f"{FILES_PATH}results.csv", delimiter=",")
startingXI_data = pd.read_csv(f"{FILES_PATH}startingXI.csv", delimiter=",")
teams_data = pd.read_csv(f"{FILES_PATH}teams.csv", delimiter=",")

In [ ]:
results_data_odds = results_data.merge(odds_data, on="MatchID", how="left")

In [ ]:
def check_similar_odds(HomeOdd, DrawOdd, AwayOdd):
    if HomeOdd==DrawOdd:
        return 1
    if AwayOdd==DrawOdd:
        return 1
    if HomeOdd==DrawOdd:
        return 1
    else:
        return 0
    
results_data_odds["SimilarOdds"] = results_data_odds.apply(lambda x: check_similar_odds(x.Home, x.Draw, x.Away), axis=1)
results_data_odds[results_data_odds["SimilarOdds"]==1]

In [ ]:
def encode_odds(HomeOdd, DrawOdd, AwayOdd):
    odds_values = np.array([HomeOdd, DrawOdd, AwayOdd])
    return np.where(odds_values == odds_values.min())[0]

In [ ]:
results_data_odds["OddEncoded"] = results_data_odds.apply(lambda x: encode_odds(x.Home, x.Draw, x.Away), axis=1)

In [ ]:
def encode_results(HomeScore, AwayScore):
    if HomeScore>AwayScore:
        return 0
    elif HomeScore<AwayScore:
        return 2 
    else:
        return 1

In [ ]:
results_data_odds["ResultEncoded"] = results_data_odds.apply(lambda x: encode_results(x.HomeScore, x.AwayScore), axis=1)

In [ ]:
results_data_odds["Upset"] = results_data_odds.apply(lambda x: 0 if x.ResultEncoded in x.OddEncoded else 1, axis=1)

In [ ]:
results_data_odds[results_data_odds["Upset"]==1]

results_data_odds["WinningOdd"] = results_data_odds.apply(lambda x: [x.Home, x.Draw, x.Away][int(x.ResultEncoded)], axis = 1)

In [ ]:
results_data_odds[results_data_odds["WinningOdd"] == results_data_odds[results_data_odds["Upset"]==1]["WinningOdd"].max()]